In [19]:
try:
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
except:
    pass

import cv2
import numpy as np
import matplotlib.pyplot as plt
from tracker import tracker

cap = cv2.VideoCapture("/home/abhinav/Desktop/ENPM673/Project2/data/challenge_video.mp4")
K = np.array([[  1.15422732e+03,0.00000000e+00,6.71627794e+02],
              [  0.00000000e+00,1.14818221e+03,3.86046312e+02],
              [  0.00000000e+00,0.00000000e+00,1.00000000e+00]])
dist = np.array([ -2.42565104e-01,-4.77893070e-02,  -1.31388084e-03,  -8.79107779e-05,
    2.20573263e-02])
def window_mask(width, height, img_ref, center, level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height), max(0,int(center-width)):min(int(center+width),img_ref.shape[1])] = 1
    return output
def adjust_gamma(image, gamma=1.0):
	# build a lookup table mapping the pixel values [0, 255] to
	# their adjusted gamma values
    lookUpTable = np.empty((1,256), np.uint8)
    for i in range(256):
        lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma) * 255.0, 0, 255)
    res = cv2.LUT(image, lookUpTable)
    return res
def preprocessing(frame):
    th = np.median(frame[int(frame.shape[0]/2):,:,:])
#     frame = adjust_gamma(frame,gamma=1.5)
#     undist = cv2.undistort(frame,K,dist)
#     blur = cv2.bilateralFilter(undist,9,75,75)
#     median = cv2.medianBlur(blur,5)
    
    
#     gray = cv2.cvtColor(median, cv2.COLOR_BGR2HLS)
#     s_channel = gray[:,:,2]
#     l_channel = gray[:,:,1]
    
    #Create mask for S Channel
    if th<60:
        s_channel = adjust_gamma(frame,gamma=25.0)
        undist = cv2.undistort(frame,K,dist)
        blur = cv2.bilateralFilter(undist,9,75,75)
        median = cv2.medianBlur(blur,5)


        gray = cv2.cvtColor(median, cv2.COLOR_RGB2LAB)
        s_channel = gray[:,:,2]
        s_channel = cv2.Sobel(s_channel,cv2.CV_64F,1,0)
        s_channel = np.absolute(s_channel)
        s_channel = np.uint8(255*s_channel/np.max(s_channel))
        l_channel = gray[:,:,0]
        s_min = 30
        s_max = 250

        l_min = 170
        l_max = 255
        
        mask1 = np.zeros([s_channel.shape[0],s_channel.shape[1]])
        mask1[(s_channel>=s_min)&(s_channel<=s_max)]=1
        
        mask2 = np.zeros([l_channel.shape[0],l_channel.shape[1]])
        mask2[(l_channel>=l_min)&(l_channel<=l_max)]=1
    
    elif th>130:
        s_channel = adjust_gamma(frame,gamma=1)
        undist = cv2.undistort(frame,K,dist)
        blur = cv2.bilateralFilter(undist,9,75,75)
        median = cv2.medianBlur(blur,5)


        gray = cv2.cvtColor(median, cv2.COLOR_BGR2LAB)
        s_channel = gray[:,:,2]
        s_channel = gray[:,:,2]
        s_channel = cv2.Sobel(s_channel,cv2.CV_64F,1,0)
        s_channel = np.absolute(s_channel)
        s_channel = np.uint8(255*s_channel/np.max(s_channel))
        l_channel = gray[:,:,0]
        s_min = 20
        s_max = 250
        
        l_min = 220
        l_max = 255
        
        mask1 = np.zeros([s_channel.shape[0],s_channel.shape[1]])
        mask1[(s_channel>=s_min)&(s_channel[1]<=s_max)]=1
        
        
        mask2 = np.zeros([l_channel.shape[0],l_channel.shape[1]])
        mask2[(l_channel>=l_min)&(l_channel<=l_max)]=1
    else:
        s_channel = adjust_gamma(frame,gamma=1.5)
        undist = cv2.undistort(frame,K,dist)
        blur = cv2.bilateralFilter(undist,9,75,75)
        median = cv2.medianBlur(blur,5)


        gray = cv2.cvtColor(median, cv2.COLOR_BGR2LAB)
        s_channel = gray[:,:,2]
        s_channel = cv2.Sobel(s_channel,cv2.CV_64F,1,0)
        s_channel = np.absolute(s_channel)
        s_channel = np.uint8(255*s_channel/np.max(s_channel))
        l_channel = gray[:,:,0]
        s_min = 30
        s_max = 250
        
        l_min = 190
        l_max = 255
        
        mask1 = np.zeros([s_channel.shape[0],s_channel.shape[1]])
        mask1[(s_channel>=s_min)&(s_channel[1]<=s_max)]=1
        
        mask2 = np.zeros([l_channel.shape[0],l_channel.shape[1]])
        mask2[(l_channel>=l_min)&(l_channel<=l_max)]=1
    
    combined = np.zeros([frame.shape[0],frame.shape[1]])
    combined[(mask1==1)|(mask2==1)]=1
    mask = np.uint8(255*combined/np.max(combined))
#     mask = cv2.inRange(median,np.array([0,190,80]),np.array([255,255,150]))
    seg = cv2.bitwise_and(frame,median,mask=mask)
    return seg,combined,th


while(True):
    ret,frame = cap.read()

    seg,combined,th = preprocessing(frame)
    #Extract the region of interest and warp to get bird's eye view
    h = frame.shape[0]
    w = frame.shape[1]
#     src = np.float32([[w-(0.5-0.08/2),h*0.62],[w*(0.5+0.08/2),h*0.62],[w*(0.5+0.76/2),h*0.935],[w*(0.5-0.76/2),h*0.935]])
#     dst = np.array([[0,0],[400,0],[400,600],[0,400]])
    src = np.array([[375,480],[905,480],[1811,685],[-531,685]])
#     dst = np.float32([[w*0.25,0],[0.75*w,0],[0.75*w,h],[0.25*w,h]])
#     src = np.array([[580,450],[160,h],[1150,h],[740,450]])
    dst = np.array([[100,0],[w,0],[w,h-100],[100,h-100]])
    H,flag = cv2.findHomography(src,dst)
    out= cv2.warpPerspective(seg,H,(w,h-100))
    out_gray = cv2.cvtColor(out,cv2.COLOR_BGR2GRAY)
    ret,thresh1 = cv2.threshold(out_gray,100,255,cv2.THRESH_BINARY)

#     font = cv2.FONT_HERSHEY_SIMPLEX
#     cv2.putText(out,str(th),(10,500), font, 4,(255,0,0),2,cv2.LINE_AA)
# #     cv2.putText(l_channel,str(th),(10,500), font, 4,(255,0,0),2,cv2.LINE_AA)
#     cv2.putText(combined,str(th),(10,500), font, 4,(255,0,0),2,cv2.LINE_AA)
    ###########################################
    
    
    window_width = 20
    window_height = 50
    curve_centers = tracker(Mywindow_width=window_width, Mywindow_height=window_height, Mymargin = 25, My_ym = 10/720, My_xm = 4/384, Mysmooth_factor=15)
    window_centroids = curve_centers.find_window_centroids(thresh1)
    # Points used to draw all the left and right windows
    l_points = np.zeros([thresh1.shape[0],thresh1.shape[1]])
    r_points = np.zeros([thresh1.shape[0],thresh1.shape[1]])
        
    # points used to find the right & left lanes
    rightx = []
    leftx = []

    # Go through each level and draw the windows 
    for level in range(0,len(window_centroids)/2):
        # Window_mask is a function to draw window areas
        # Add center value found in frame to the list of lane points per left, right
        leftx.append(window_centroids[level][0])
        rightx.append(window_centroids[level][1])

        l_mask = window_mask(window_width,window_height,thresh1,window_centroids[level][0],level)
        r_mask = window_mask(window_width,window_height,thresh1,window_centroids[level][1],level)
        # Add graphic points from window mask here to total pixels found 
        l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
        r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

    # Draw the results
    template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
    zero_channel = np.zeros_like(template) # create a zero color channel
    template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
    warpage = np.array(cv2.merge((thresh1,thresh1,thresh1)),np.uint8) # making the original road pixels 3 color channels
    result = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the original road image with window results

    
    
    
    
    
    cv2.imshow('s',result)
    cv2.imshow('l',thresh1)
#     cv2.imshow('test',l_channel)
#     cv2.imshow('mask',mask)
    if cv2.waitKey(1)& 0xff==ord('q'):
        cv2.destroyAllWindows()
        break

In [18]:
for level in range(int(len(window_centroids)/2),len(window_centroids)):
    print level

6
7
8
9
10
11


In [5]:
def ransac_polyfit(x, y, order=3, n=20, k=100, t=0.1, d=100, f=0.8):
    # Thanks https://en.wikipedia.org/wiki/Random_sample_consensus

    # n – minimum number of data points required to fit the model
    # k – maximum number of iterations allowed in the algorithm
    # t – threshold value to determine when a data point fits a model
    # d – number of close data points required to assert that a model fits well to data
    # f – fraction of close data points required

    besterr = np.inf
    bestfit = None
    for kk in xrange(k):
        maybeinliers = np.random.randint(len(x), size=n)
        maybemodel = np.polyfit(x[maybeinliers], y[maybeinliers], order)
        alsoinliers = np.abs(np.polyval(maybemodel, x)-y) < t
        if sum(alsoinliers) > d and sum(alsoinliers) > len(x)*f:
            bettermodel = np.polyfit(x[alsoinliers], y[alsoinliers], order)
            thiserr = np.sum(np.abs(np.polyval(bettermodel, x[alsoinliers])-y[alsoinliers]))
            if thiserr < besterr:
                bestfit = bettermodel
                besterr = thiserr
    return bestfit

In [9]:
left_laneFit.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [18]:
np.argmax(histogram[600:])

271

In [14]:
a=[9,2,3,4,5,6,7,8]
b = []
a.append(b)
# max(a)

In [41]:
a=[9,[2,345],3,4,5,6,7,8,[]]
b= min(a)
b

3

In [30]:
b = []
for i in range(len(a)):
    if a==[]:
        pass
    else:
        b.append(a[i])

In [31]:
b

[9, 2, 3, 4, 5, 6, 7, 8, []]

In [7]:
#Testing another pipeline

def binary_transform(img, sobel_kernel=3, sc_thresh=(170, 255), sx_thresh=(20, 100)):
    """ Given an input image it convert the image color space
        from RGB to HLS, tekes only the S channel and applies
        Sobel thresholding along X axis.

        Returns: a color binary image with only those pixels
        set to 1 iff the Sobel values were within the sx_thresh
        and the S channel values were within the sc_thresh.
    """
    
    img = np.copy(img)

    # Convert to HSV color space and separate the S channel
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    h_channel = hsv[:,:,0]
    l_channel = hsv[:,:,1]
    s_channel = hsv[:,:,2]

    # Use the s_channel
    channel = s_channel
    
    # Sobel x and y
    sobel_x = cv2.Sobel(channel, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobel_y = cv2.Sobel(channel, cv2.CV_64F, 0, 1, ksize=sobel_kernel)

    # Absolute derivative in x and y
    sobel_x_abs = np.absolute(sobel_x)
    sobel_y_abs = np.absolute(sobel_y)

    # Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel_x = np.uint8(255*sobel_x_abs/np.max(sobel_x_abs))

    # Threshold x gradient
    sx_binary = np.zeros([scaled_sobel_x.shape[0],scaled_sobel_x.shape[1]])
    sx_binary[(scaled_sobel_x >= sx_thresh[0]) & (scaled_sobel_x <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros([s_channel.shape[0],s_channel.shape[1]])
    s_binary[(channel >= sc_thresh[0]) & (channel <= sc_thresh[1])] = 1

    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.dstack(( np.zeros([sx_binary.shape[0],sx_binary.shape[1]]), sx_binary, s_binary))

    # Create a binary flat image.
    flat_binary = np.zeros([sx_binary.shape[0],sx_binary.shape[1]])
    flat_binary[(sx_binary == 1) | (s_binary == 1)] = 1

    return flat_binary


In [17]:
def to_real_world_space(image, warped, Minv, left_fitx, right_fitx, ploty):
    # Create an image to draw the lines on
    color_warp = np.zeros_like(warped).astype(np.uint8)
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0]))

    # Combine the result with the original image
    result = cv2.addWeighted(image, 1, newwarp, 0.3, 0)
    return result

Minv = np.linalg.inv(M)
final_output = to_real_world_space(img, img_lines, Minv, l_fit, r_fit, ploty)
display_save_images([img, final_output], ['Original', 'Pipeline Result'], 'final_result')

NameError: name 'M' is not defined

In [ ]:
# Compute curve radius    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    #left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    #right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad =  ((1 + (2*left_fit_cr[0] *y_eval*ym_per_pix + left_fit_cr[1])**2) **1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

    # Compute car position
    m_car = binary_warped.shape[1] / 2
    m_lane = (left_fitx[0] + right_fitx[0]) / 2
    offset_right_from_center_m = (m_lane-m_car)*xm_per_pix
    
    # Now our radius of curvature is in meters
    avg_radius_meters = np.mean([left_curverad, right_curverad])
    return out_img, avg_radius_meters, offset_right_from_center_m, left_fitx, right_fitx, ploty

img_lines, r_meters, right_from_center_m, l_fit, r_fit, ploty = fit_lines(warped)
display_save_images([warped, img_lines], ['Original', 'Lines - R: {:.0f} m'.format(r_meters)], 'lines')


In [9]:
import numpy as np
import cv2
class tracker():
    
    def __init__(self, Mywindow_width, Mywindow_height, Mymargin, My_ym = 1, My_xm = 1, Mysmooth_factor = 15):
        #list that stores all the past (left,right,center) set values used for smoothing the output
        self.recent_centers = []

        # The window pixel width of the center values, used to count pixels inside center windows to determine curve values
        self.window_width = Mywindow_width

        # The window pixel height of the center values, used to count pixels inside center windows to determine curve values
        # breaks the image into vertical levels
        self.window_height = Mywindow_height

        # The pixel distance in both directions to slide (left_window + right_window) template for searching
        self.margin = Mymargin

        self.ym_per_pix = My_ym # meters per pixel in vertical axis

        self.xm_per_pix = My_xm # meters per pixel in horizontal axis

        self.smooth_factor = Mysmooth_factor

        # Main tracking function for finding & storing the lane segment positions
    def find_window_centroids(self, warped):

        window_width = self.window_width
        window_height = self.window_height
        margin = self.margin

        window_centroids = [] # store the (left,right) window centroid positions per level
        window = np.ones(window_width) # create our window template that we will use for convolutions

        # first find the two starting positions for the left & right lane by using np.sum to get the vertical image slice
        # and then np.convolve the vertical image slice with the window template

        #sum quarter bottom of image to get slice, could use a different ratio
        l_sum = np.sum(warped[int(3*warped.shape[0]/4):,:int(warped.shape[1]/2)], axis=0)
        l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
        r_sum = np.sum(warped[int(3*warped.shape[0]/4):,int(warped.shape[1]/2):], axis=0)
        r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(warped.shape[1]/2)

        # Add what we found for the 1st layer
        window_centroids.append((l_center,r_center))
        minPix = 30
        # Go thru each layer looking for max pixel locations
        for level in range(1,(int)(warped.shape[0]/window_height)):
            # convolve the window into the vertical slice of the image
            image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_height):int(warped.shape[0]-level*window_height),:], axis=0)
            conv_signal = np.convolve(window, image_layer)
            # Find the best left centroid by using past left center as a reference
            # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
            offset = window_width/2
            l_min_index = int(max(l_center+offset-margin,0))
            l_max_index = int(min(l_center+offset+margin,warped.shape[1]))
            l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
            # Find the best right centroid by using past right center as a reference
            r_min_index = int(max(r_center+offset-margin,0))
            r_max_index = int(min(r_center+offset+margin,warped.shape[1]))
            r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
            # Add what we found for that layer
            window_centroids.append((l_center,r_center))
            #the new window form their median position
            if len(left_inds)>minPix:
                left_curr = np.int(np.mean(x[left_ind]))
            if len(right_inds)>minPix:
                right_curr = np.int(np.mean(x[right_ind]))

        self.recent_centers.append(window_centroids)
        # return averaged values of the line centers, helps to keep the markers from sliding too much
        return np.average(self.recent_centers[-self.smooth_factor:], axis=0)

In [ ]:
#Histogram
    histogram = np.sum(thresh1[thresh1.shape[0]/2:,:],axis = 0)
    plt.plot(histogram)
    mid = np.int((dst[0][0]+dst[1][0])/2)
#     mid = np.int(histogram.shape[0]/2)
    left_lane = np.argmax(histogram[:mid])
    right_lane = np.argmax(histogram[mid:])+mid
    
    wndw = 9
    
    wndw_ht = np.int(thresh1.shape[0]/wndw)
    
    nonzero = thresh1.nonzero()
    x = np.array(nonzero[0])
    y = np.array(nonzero[1])
    
    left_curr = left_lane
    right_curr = right_lane
    
    margin = 50   
    minPix = 10
    
    #Find indices for left and right pixels
    left_inds = []
    right_inds = []
    
    for win in range(wndw):
        y_min = thresh1.shape[0] - (win+1)*wndw_ht
        y_max = thresh1.shape[0] - (win)*wndw_ht
        x_min_left = left_curr - margin
        x_max_left = left_curr + margin
        x_min_right = right_curr - margin
        x_max_right = right_curr + margin
        
        cv2.rectangle(out,(x_min_left,y_min),(x_max_left,y_max),(255,0,255),2)
        cv2.rectangle(out,(x_min_right,y_min),(x_max_right,y_max),(255,0,0),2)
        
        left_ind = ((y>=y_min)&(y<=y_max)&(x>=x_min_left)&(x<=x_max_left)).nonzero()[0]
        right_ind = ((y>=y_min)&(y<=y_max)&(x>=x_min_right)&(x<=x_max_right)).nonzero()[0]
#         if len(left_ind) !=0:
        left_inds.append(left_ind)
#         if len(right_ind) !=0:
        right_inds.append(right_ind)
        #If no. of white pixels > minPix then start
        
        #the new window form their median position
        if len(left_inds)>minPix:
            left_curr = np.int(np.mean(x[left_ind]))
        if len(right_inds)>minPix:
            right_curr = np.int(np.mean(x[right_ind]))
    
    left_inds = np.concatenate(left_inds)
    right_inds = np.concatenate(right_inds)
    sorted(left_inds)
    sorted(right_inds)
#     x_left = [x[int(min(left_inds))],x[int(max(left_inds))],x[int(np.median(left_inds))]]
#     y_left = [y[int(min(left_inds))],y[int(max(left_inds))],y[int(np.median(left_inds))]]
#     x_right = [x[int(min(right_inds))],x[int(max(right_inds))],x[int(np.median(right_inds))]]
#     y_right = [y[int(min(right_inds))],y[int(max(right_inds))],y[int(np.median(right_inds))]] 
    x_left = x[left_inds]
    x_right = x[right_inds]
    y_left = y[left_inds]
    y_right = y[right_inds]
    print(x_right)
    ##################################################
    #Fit a Curve using RANSAC
#     left_laneFit = ransac_polyfit(x_left, y_left, order=3, n=20, k=100, t=0.1, d=100, f=0.8)
#     right_laneFit = ransac_polyfit(x_right, y_right, order=3, n=20, k=100, t=0.1, d=100, f=0.8)
    plotx = np.linspace(0,thresh1.shape[0]-1,thresh1.shape[0]/2)
    if len(x_left)!=0:    
        left_laneFit =  np.polyfit(x_left,y_left,2)
        left_fit = left_laneFit[0]*plotx**2 + left_laneFit[1]*plotx + left_laneFit[2]
        pts_left = np.vstack((left_fit,plotx)).astype(np.int32).T
        cv2.polylines(out,  [pts_left],  False,  (255, 0, 0),  1)
    if len(x_right)!=0:
        right_laneFit = np.polyfit(x_right,y_right,2)
        right_fit = right_laneFit[0]*plotx**2 + right_laneFit[1]*plotx + right_laneFit[2]
        pts_right = np.vstack((right_fit,plotx)).astype(np.int32).T
        cv2.polylines(out,  [pts_right],  False,  (0, 0, 255),  1)
    
    
    #Plot the generated curves
#      left_fit = left_laneFit[0]*plotx**2 + left_laneFit[1]*plotx + left_laneFit[2]
    
    
    
    

